In [0]:
%tensorflow_version 1.x
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization as BN
from keras.layers import GaussianNoise as GN
from keras.optimizers import SGD

from keras.callbacks import LearningRateScheduler as LRS
from keras.preprocessing.image import ImageDataGenerator

TensorFlow 1.x selected.


Using TensorFlow backend.


In [0]:

batch_size = 100
num_classes = 10
epochs = 75


#### LOAD AND TRANSFORM
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

print(x_train.shape)
print(x_test.shape)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


## DEFINE A DATA AUGMENTATION GENERATOR

datagen = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    rotation_range = 15,
    horizontal_flip=True)


170500096/170498071 [==============================] - 6s 0us/step
(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [0]:


## DEF A BLOCK CONV + BN + GN + MAXPOOL
def CBGN(model,filters,num_blocs = 0,ishape=0,C=False):

  for i in range(num_blocs):
  
    if (ishape!=0):
      model.add(Conv2D(filters, (3, 3), padding='same',
                  input_shape=ishape))
    else:
      if i == num_blocs-1 and C:
        model.add(Conv2D(filters, (1, 1), padding='same'))
      else:
        model.add(Conv2D(filters, (3, 3), padding='same'))

    model.add(BN())
    model.add(GN(0.3))
    model.add(Activation('relu'))


  model.add(MaxPooling2D(pool_size=(2, 2)))

  return model

def create_vgg(parameters = [1,1,2,2,2], C = False):
    ## DEF NN TOPOLOGY  
    model = Sequential()
    model=CBGN(model,64,ishape=x_train.shape[1:],num_blocs=parameters[0])
    model=CBGN(model,128,num_blocs=parameters[1])
    model=CBGN(model,245,num_blocs=parameters[2],C = C)
    model=CBGN(model,512,num_blocs=parameters[3],C = C)
    model=CBGN(model,512,num_blocs=parameters[4],C = C)

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(512))
    model.add(Activation('relu'))

    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    return model 

def create_vgg_by_type(type= "A"):
  if type == "A":
    return create_vgg(parameters = [1,1,2,2,2], C = False)
  elif type == "B":
    return create_vgg(parameters = [2,2,2,2,2], C = False)
  elif type == "C":
    return create_vgg(parameters = [2,2,3,3,3], C = True)
  elif type == "D":
    return create_vgg(parameters = [2,2,3,3,3], C = False)
  elif type == "E":
    return create_vgg(parameters = [2,2,4,4,4], C = False)
  

In [0]:

model = create_vgg_by_type(type= "A")
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
gaussian_noise_1 (GaussianNo (None, 32, 32, 64)        0         
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       73856     
______________________________________

In [0]:


## OPTIM AND COMPILE
opt = SGD(lr=0.1, decay=1e-6)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

# DEFINE A LEARNING RATE SCHEDULER
def scheduler(epoch):
    if epoch < 25:
        return .1
    elif epoch < 50:
        return 0.01
    else:
        return 0.001

set_lr = LRS(scheduler)


## TRAINING with DA and LRA
history=model.fit_generator(datagen.flow(x_train, y_train,batch_size=batch_size),
                            steps_per_epoch=len(x_train) / batch_size, 
                            epochs=epochs,
                            validation_data=(x_test, y_test),
                            callbacks=[set_lr],
                            verbose=1)


## TEST
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])









Epoch 1/75
500/500 [==============================] - 43s 86ms/step - loss: 1.7058 - accuracy: 0.3730 - val_loss: 1.6239 - val_accuracy: 0.4131
Epoch 2/75
500/500 [==============================] - 35s 71ms/step - loss: 1.3764 - accuracy: 0.4999 - val_loss: 2.3430 - val_accuracy: 0.3666
Epoch 3/75
500/500 [==============================] - 35s 70ms/step - loss: 1.2081 - accuracy: 0.5639 - val_loss: 1.6355 - val_accuracy: 0.5027
Epoch 4/75
500/500 [==============================] - 35s 71ms/step - loss: 1.0776 - accuracy: 0.6175 - val_loss: 1.4430 - val_accuracy: 0.5543
Epoch 5/75
500/500 [==============================] - 36s 71ms/step - loss: 0.9917 - accuracy: 0.6484 - val_loss: 2.0895 - val_accuracy: 0.4225
Epoch 6/75
500/500 [==============================] - 36s 72ms/step - loss: 0.9195 - accuracy: 0.6751 - val_loss: 0.8906 - val_accuracy: 0.6936
Epoch 7/75
500/500 [==============================] - 35s 70ms/step - loss: 0.8684 - accuracy: 0.6954 - val_loss: 1.0739 - val_accuracy